In [ ]:
from autumn.infrastructure.remote import springboard
from autumn.projects.sm_covid2.common_school.runner_tools import run_full_analysis 

### Define task function

In [ ]:
N_CHAINS = 4
ISO3 = "FRA"
ANALYSIS = "main"

def remote_full_analysis_task(bridge: springboard.task.TaskBridge, iso3: str = 'FRA', analysis: str = "main"):
    
    import multiprocessing as mp
    mp.set_start_method('forkserver')

    bridge.logger.info(f"Running full analysis for {iso3}. Analysis type: {analysis}.")

    idata, uncertainty_df, diff_quantiles_df = run_full_analysis(
        iso3,
        analysis=analysis, 
        opti_params={'warmup_iterations': 200, 'search_iterations': 500},
        mcmc_params={'draws': 500, 'tune': 100, 'cores': N_CHAINS, 'chains': N_CHAINS},
        full_run_params={'samples': 100, 'burn_in': 0},
        output_folder=bridge.out_path,
        logger=bridge.logger
    )
   
    bridge.logger.info("Full analysis complete")

In [ ]:
# Get a compute machine to do.. some computation.
mspec = springboard.EC2MachineSpec(N_CHAINS, 4, "compute")

# Wrap our function; the first argument is supplied by the runner, the rest must be kwargs

task_kwargs = {
    "iso3": ISO3,
    "analysis": ANALYSIS
}

tspec = springboard.TaskSpec(remote_full_analysis_task, task_kwargs)

In [ ]:
run_path = springboard.launch.get_autumn_project_run_path("school_project", ISO3, f"{ANALYSIS}_first_test")
run_path

In [ ]:
# This should be fairly resilient to failure, however if you do receive any kind of error or Exception here,
# please tell David ASAP!
# It is possible you will get some "SSH connection will waiting, retrying" messages; please also report these
# - they're not a 'failture', just AWS being a bit slow...

runner = springboard.launch.launch_synced_autumn_task(tspec, mspec, run_path)

In [ ]:
runner.instance

In [ ]:
# Use 'tail' to report the last (n, default=10) lines of command output
# This is only valid while the task is actually running, and is included here to demonstrate its use in debugging 
# Typical runs would just use the wait method instead (see cell below)

print(runner.tail())

In [ ]:
print(runner.top("+%CPU"))

In [ ]:
runner.s3.get_status()

In [ ]:
runner.wait()

In [ ]:
# This is the complete 'command line' output from the run
print(runner.get_iodump())

In [ ]:
# This should print out a log containing the text from our wrapped task...
print(runner.get_log())

In [ ]:
from autumn.core.runs import ManagedRun

In [ ]:
mr = ManagedRun(run_path)

In [ ]:
mr.remote.list_contents()

In [ ]:
for f in mr.remote.list_contents():
    mr.remote.download(f)

In [ ]:
print(open(mr.local_path / ".taskmeta/task_spec.yml").read())